# Notebook #1


### Name: Tulsi Patel


### Date: 11.11.2021


### PID: 730392259


### Goal: Find average firing rate for Brain Observatory, WT, Male, VISl, Static Gratings (SG) and Drifting Gratings (DG). 

*There are no Female SST VISp and VISl sessions. There are no Female WT/WT VISp and VISl sessions. (Disregard the Jupyter notebooks labeled "Female WT". They are not actually WT/WT. They are hybrids. 

## Protocol

### Starting code. 

In [1]:
import os

import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from allensdk.brain_observatory.ecephys.ecephys_session import (
    EcephysSession
)
from allensdk.brain_observatory.ecephys.visualization import plot_mean_waveforms, plot_spike_counts, raster_plot
from allensdk.brain_observatory.visualization import plot_running_speed

# tell pandas to show all columns when we display a DataFrame
pd.set_option("display.max_columns", None)

In [2]:
data_directory = '/Users/tulsipatel/local1/ecephys_cache_dir' 
# must be updated to a valid directory in your filesystem

manifest_path = os.path.join(data_directory, "manifest.json")

In [3]:
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

In [4]:
units = cache.get_units()

In [5]:
sessions = cache.get_session_table()


print('Total number of sessions: ' + str(len(sessions)))

sessions.head()

Total number of sessions: 58


,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
715093703,2019-10-03T00:00:00Z,699733581,brain_observatory_1.1,118.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,884,2219,6,"[CA1, VISrl, nan, PO, LP, LGd, CA3, DG, VISl, ..."
719161530,2019-10-03T00:00:00Z,703279284,brain_observatory_1.1,122.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,755,2214,6,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
721123822,2019-10-03T00:00:00Z,707296982,brain_observatory_1.1,125.0,M,Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,444,2229,6,"[MB, SCig, PPT, NOT, DG, CA1, VISam, nan, LP, ..."
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
737581020,2019-10-03T00:00:00Z,718643567,brain_observatory_1.1,108.0,M,wt/wt,568,2218,6,"[grey, VISmma, nan, VISpm, VISp, VISl, VISrl]"


In [6]:
filtered_sessions = sessions[(sessions.sex == 'M') & \
                             (sessions.full_genotype.str.find('wt/wt') > -1) & \
                             (sessions.session_type == 'brain_observatory_1.1') & \
                             (['VISl' in acronyms for acronyms in 
                               sessions.ecephys_structure_acronyms])]

filtered_sessions.head()

,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
737581020,2019-10-03T00:00:00Z,718643567,brain_observatory_1.1,108.0,M,wt/wt,568,2218,6,"[grey, VISmma, nan, VISpm, VISp, VISl, VISrl]"
739448407,2019-10-03T00:00:00Z,716813543,brain_observatory_1.1,112.0,M,wt/wt,625,2221,6,"[grey, VISam, nan, VIS, VISp, VISl, VISrl]"
742951821,2019-10-03T00:00:00Z,723627604,brain_observatory_1.1,120.0,M,wt/wt,893,2219,6,"[VISal, nan, grey, VISl, VISrl, VISp, VISpm, VIS]"
743475441,2019-10-03T00:00:00Z,722882755,brain_observatory_1.1,121.0,M,wt/wt,553,2225,6,"[LP, LGd, HPF, DG, CA3, CA1, VISrl, nan, PP, P..."


In [7]:
session_id = filtered_sessions.index.values[2]

session = cache.get_session_data(session_id)

In [8]:
units = session.units

print('Total number of units: ' + str(len(units)))

units.head()

Total number of units: 625


,peak_channel_id,waveform_halfwidth,waveform_amplitude,waveform_velocity_below,L_ratio,snr,amplitude_cutoff,waveform_spread,nn_miss_rate,waveform_repolarization_slope,presence_ratio,isi_violations,nn_hit_rate,local_index_unit,d_prime,max_drift,waveform_PT_ratio,cluster_id,isolation_distance,silhouette_score,waveform_velocity_above,cumulative_drift,firing_rate,waveform_duration,waveform_recovery_slope,c50_dg,area_rf,fano_dg,fano_fl,fano_ns,fano_rf,fano_sg,f1_f0_dg,g_dsi_dg,g_osi_dg,g_osi_sg,azimuth_rf,mod_idx_dg,p_value_rf,pref_sf_sg,pref_tf_dg,run_mod_dg,run_mod_fl,run_mod_ns,run_mod_rf,run_mod_sg,pref_ori_dg,pref_ori_sg,run_pval_dg,run_pval_fl,run_pval_ns,run_pval_rf,run_pval_sg,elevation_rf,pref_image_ns,pref_phase_sg,firing_rate_dg,firing_rate_fl,firing_rate_ns,firing_rate_rf,firing_rate_sg,on_off_ratio_fl,time_to_peak_ns,pref_sf_multi_sg,pref_tf_multi_dg,pref_ori_multi_dg,pref_ori_multi_sg,pref_phase_multi_sg,image_selectivity_ns,pref_image_multi_ns,lifetime_sparseness_dg,lifetime_sparseness_fl,lifetime_sparseness_ns,lifetime_sparseness_rf,lifetime_sparseness_sg,probe_vertical_position,probe_horizontal_position,probe_id,channel_local_index,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
951808989,850211269,0.164824,100.057815,0.000000,0.036386,2.840270,0.026085,30.0,0.034483,0.413804,0.99,0.024853,0.938000,5,3.335647,16.27,0.346582,5,95.979386,0.055628,0.000000,137.52,31.839506,0.302178,-0.071154,NaN,1000.0,1.652282,0.997110,2.086087,0.890021,2.062046,0.193774,0.033770,0.016799,0.008807,40.000,0.633406,0.000,0.08,1.0,NaN,NaN,-0.229678,NaN,-0.195122,225.0,0.0,NaN,NaN,0.187761,NaN,0.694848,-15.000,4911,0.50,35.170606,37.595905,28.003696,31.890527,30.338949,NaN,0.1795,False,False,False,False,False,0.199017,False,0.004952,0.027629,0.006559,0.013269,0.006948,140,11,757968489,13,8.0,grey,NaN,NaN,NaN,probeA,See electrode locations,29999.964361,1249.998515,True
951808976,850211269,0.164824,120.798210,0.000000,0.070177,3.305557,0.043860,20.0,0.030250,0.546024,0.99,0.008103,0.904667,4,3.853885,13.74,0.684731,4,69.787442,0.204123,NaN,200.04,28.346921,0.260972,-0.153623,NaN,800.0,3.576891,0.909264,1.623596,1.236158,1.414272,0.105344,0.020643,0.011077,0.012017,48.750,1.310929,0.000,0.02,15.0,NaN,NaN,0.170188,NaN,-0.002551,270.0,0.0,NaN,NaN,0.084907,NaN,0.993035,-18.750,5022,0.75,32.582967,23.420837,30.048144,9.436456,29.782766,NaN,0.1525,False,False,False,False,False,0.001627,False,0.004001,0.005228,0.003936,0.054303,0.003511,140,11,757968489,13,8.0,grey,NaN,NaN,NaN,probeA,See electrode locations,29999.964361,1249.998515,True
951809003,850211277,0.137353,76.722945,1.030151,0.006293,1.926873,0.023672,70.0,0.000067,0.315857,0.98,0.000000,0.769231,6,3.862677,19.92,0.189503,6,41.314878,NaN,0.343384,310.35,0.279998,0.233501,-0.019571,NaN,4100.0,0.812121,NaN,1.291429,0.800000,1.180000,1.841297,0.100145,0.169811,0.037816,52.195,0.446237,0.344,0.32,8.0,NaN,NaN,0.492308,-1.0,0.590909,315.0,120.0,NaN,NaN,0.341355,0.082097,0.130740,-9.756,4920,0.75,0.170497,0.000000,0.498351,0.129387,0.454938,NaN,0.1395,False,False,False,False,False,0.194797,True,0.191987,NaN,0.157744,0.680890,0.172531,180,11,757968489,17,8.0,grey,NaN,NaN,NaN,probeA,See electrode locations,29999.964361,1249.998515,True
951809147,850211317,0.123618,122.473455,-1.373534,0.003246,3.021903,0.003017,60.0,0.000553,0.513822,0.99,0.015365,0.985333,14,4.807625,19.06,0.527284,14,71.310975,0.116839,-0.137353,188.33,9.868341,0.260972,-0.116531,NaN,500.0,4.663087,0.981026,1.146081,1.042857,2.670968,0.363796,0.051481,0.019883,0.050015,14.000,1.007488,0.000,0.08,1.0,NaN,NaN,0.423645,NaN,0.416667,45.0,30.0,NaN,NaN,0.000116,NaN,0.040909,2.000,5001,0.25,7.448927,8.952675,17.706234,4.205067,13.782680,NaN,0.1575,False,False,Fals

In [9]:
V1_units = units[(units.ecephys_structure_acronym == 'VISl') &
                (units.isi_violations < 0.1)]

print('Total number of low-contamination V1 units: ' + str(len(V1_units)))


V1_units.head()

Total number of low-contamination V1 units: 17


,peak_channel_id,waveform_halfwidth,waveform_amplitude,waveform_velocity_below,L_ratio,snr,amplitude_cutoff,waveform_spread,nn_miss_rate,waveform_repolarization_slope,presence_ratio,isi_violations,nn_hit_rate,local_index_unit,d_prime,max_drift,waveform_PT_ratio,cluster_id,isolation_distance,silhouette_score,waveform_velocity_above,cumulative_drift,firing_rate,waveform_duration,waveform_recovery_slope,c50_dg,area_rf,fano_dg,fano_fl,fano_ns,fano_rf,fano_sg,f1_f0_dg,g_dsi_dg,g_osi_dg,g_osi_sg,azimuth_rf,mod_idx_dg,p_value_rf,pref_sf_sg,pref_tf_dg,run_mod_dg,run_mod_fl,run_mod_ns,run_mod_rf,run_mod_sg,pref_ori_dg,pref_ori_sg,run_pval_dg,run_pval_fl,run_pval_ns,run_pval_rf,run_pval_sg,elevation_rf,pref_image_ns,pref_phase_sg,firing_rate_dg,firing_rate_fl,firing_rate_ns,firing_rate_rf,firing_rate_sg,on_off_ratio_fl,time_to_peak_ns,pref_sf_multi_sg,pref_tf_multi_dg,pref_ori_multi_dg,pref_ori_multi_sg,pref_phase_multi_sg,image_selectivity_ns,pref_image_multi_ns,lifetime_sparseness_dg,lifetime_sparseness_fl,lifetime_sparseness_ns,lifetime_sparseness_rf,lifetime_sparseness_sg,probe_vertical_position,probe_horizontal_position,probe_id,channel_local_index,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
951803445,850214919,NaN,117.286455,0.000000,0.002371,1.644926,0.065034,70.0,0.000335,0.039062,0.98,0.000000,0.796610,377,3.134196,40.69,1.672628,383,50.265686,-0.054956,1.295047,143.80,0.340099,1.469682,-0.041879,NaN,400.0,5.260606,1.442051,1.143636,2.233333,1.079787,0.870892,0.081495,0.101639,0.122633,35.000,1.970524,0.036,0.32,1.0,NaN,NaN,0.247863,NaN,0.475000,45.0,90.0,NaN,NaN,7.513963e-01,NaN,5.098279e-01,25.000,5011,0.0,0.484487,0.452963,0.302234,0.101974,0.141211,NaN,0.0165,False,False,False,False,False,0.310576,False,0.235019,0.437838,0.253899,0.784870,0.429406,2060,11,757968495,205,409.0,VISl,NaN,NaN,NaN,probeD,See electrode locations,29999.916144,1249.996506,True
951801452,850214921,0.178559,153.819900,1.030151,0.011320,1.420207,0.002685,60.0,0.001223,0.467110,0.99,0.014973,0.924309,206,4.829578,44.80,0.885939,211,57.781495,0.183296,0.686767,110.90,3.286939,0.769179,-0.023668,NaN,300.0,2.543590,1.546667,0.785000,1.319540,1.931241,0.284405,0.059469,0.582304,0.472808,46.667,4.215820,0.000,0.08,2.0,NaN,NaN,-0.085763,NaN,0.310714,45.0,30.0,NaN,NaN,4.742117e-01,NaN,6.302066e-02,6.667,4982,0.5,3.079539,0.373028,6.388564,4.144760,4.653287,NaN,0.0975,False,False,False,False,False,0.650356,False,0.592797,0.150943,0.478788,0.752821,0.461196,2080,59,757968495,206,409.0,VISl,NaN,NaN,NaN,probeD,See electrode locations,29999.916144,1249.996506,True
951801512,850214925,NaN,75.366915,0.098110,0.003762,0.940315,0.020264,80.0,0.038110,0.013291,0.99,0.017603,0.968667,211,2.327240,43.26,12.792510,216,88.123620,0.085102,0.686767,117.01,11.443701,1.208710,-0.012667,NaN,4000.0,4.486778,1.426498,1.766667,0.560000,2.716414,0.471750,0.073872,0.040374,0.048841,62.250,0.680608,0.000,0.32,8.0,NaN,NaN,-0.861842,NaN,-0.871069,315.0,90.0,NaN,NaN,1.008447e-08,NaN,2.771454e-08,-3.250,4945,0.5,11.371951,11.563872,11.096704,11.837785,9.924702,NaN,0.0015,False,False,False,False,False,0.088186,False,0.023736,0.016846,0.020943,0.063837,0.027702,2100,43,757968495,208,409.0,VISl,NaN,NaN,NaN,probeD,See electrode locations,29999.916144,1249.996506,True
951801599,850214933,0.178559,98.391735,-0.343384,0.003953,1.340376,0.098342,40.0,0.001277,0.421730,0.98,0.056441,0.937255,218,4.413431,29.68,0.222142,223,60.489391,0.034377,0.000000,274.32,3.879091,0.288442,-0.054783,NaN,0.0,2.287179,0.986667,3.324751,0.933333,5.278931,1.613865,0.108951,0.116844,0.028520,NaN,1.026624,0.763,0.04,15.0,NaN,NaN,-0.235842,NaN,0.717577,270.0,150.0,NaN,NaN,5.409482e-01,NaN,4.515533e-03,NaN,4957,0.0,0.216563,0.053290,9.611054,0.0

## Find average firing rate for static gratings. 

In [10]:
print(V1_units.firing_rate_sg)

unit_id
951803445     0.141211
951801452     4.653287
951801512     9.924702
951801599     8.177555
951801718     8.142252
951801969     7.027222
951802129    13.313089
951802041     9.015493
951802218    12.307297
951802163     6.235910
951802276     8.695771
951802249     8.753721
951802316    11.988241
951802354     2.159455
951803458     3.754069
951802446     1.879033
951802942    16.331131
Name: firing_rate_sg, dtype: float64


In [11]:
static_firing_rates=V1_units.firing_rate_sg 
static_average_firing_rate = sum(V1_units.firing_rate_sg)/len(V1_units.firing_rate_sg)
print(static_average_firing_rate)

7.794084541535627


## Find average firing rate for drifting gratings. 

In [12]:
print(V1_units.firing_rate_dg)

unit_id
951803445     0.484487
951801452     3.079539
951801512    11.371951
951801599     0.216563
951801718     0.979565
951801969     5.347893
951802129    12.019523
951802041     7.547413
951802218     8.946337
951802163     5.212343
951802276     5.626407
951802249     6.466715
951802316    10.089516
951802354     2.318656
951803458     1.783337
951802446     1.317382
951802942     8.855794
Name: firing_rate_dg, dtype: float64


In [13]:
drifting_firing_rates=V1_units.firing_rate_dg 
drifting_average_firing_rate = sum(V1_units.firing_rate_dg)/len(V1_units.firing_rate_dg)
print(drifting_average_firing_rate)

5.391965978429075


## Remaining code: I was experimenting with another way to find the average firing rate. 

In [ ]:
presentation_table = session.stimulus_presentations[session.stimulus_presentations.stimulus_name == 'natural_scenes']


presentation_times = presentation_table.start_time.values
presentation_ids = presentation_table.index.values
presentation_table.head()

In [ ]:
units_of_interest = V1_units.index.values

In [ ]:
times = session.presentationwise_spike_times(
    stimulus_presentation_ids=presentation_ids,
    unit_ids=units_of_interest
)

times.head()

In [ ]:
average_firing_rate = '.0023476'

In [ ]:
length_of_time=times.time_since_stimulus_presentation_onset

In [ ]:
print(length_of_time.index.values[4])